In [1]:
# Imports
import os
import sys
import pickle
import random
from typing import List, Dict, Callable
from collections import defaultdict, Counter

import csv
import gzip
import h5py
import shutil
import zipfile
import pydicom
import numpy as np


# Keras imports
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ThresholdedReLU, Lambda, Reshape, LayerNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, add, concatenate
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten, LSTM, RepeatVector
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D, UpSampling1D, UpSampling2D, UpSampling3D, MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D, Layer
from tensorflow.keras.layers import SeparableConv1D, SeparableConv2D, DepthwiseConv2D


%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

from ml4cvd.defines import StorageType
from ml4cvd.arguments import parse_args, TMAPS, _get_tmap
from ml4cvd.TensorMap import TensorMap, Interpretation
from ml4cvd.tensor_generators import test_train_valid_tensor_generators
from ml4cvd.models import train_model_from_generators, make_multimodal_multitask_model, _inspect_model, train_model_from_generators, make_hidden_layer_model
from ml4cvd.recipes import test_multimodal_multitask, train_multimodal_multitask, saliency_maps

# Constants
HD5_FOLDER = '/mnt/disks/brains-all-together/2020-02-11/'
MODEL_FOLDER = './models/'

In [2]:
sys.argv = ['train', 
            '--tensors', HD5_FOLDER, 
            '--input_tensors', 't1',
            '--output_tensors', 'sex',
            '--training_steps', '96',
            '--validation_steps', '24',
            '--test_steps', '12',
            '--batch_size', '1'
           ]

args = parse_args()

2020-04-13 13:34:37,718 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/no_id/log_2020-04-13_13-34_0.log.
2020-04-13 13:34:37,719 - arguments:241 - INFO - Command Line was: 
./scripts/tf.sh train --tensors /mnt/disks/brains-all-together/2020-02-11/ --input_tensors t1 --output_tensors sex --training_steps 96 --validation_steps 24 --test_steps 12 --batch_size 1

2020-04-13 13:34:37,720 - arguments:242 - INFO - Total TensorMaps: 474 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, anneal_max=2.0, anneal_rate=0.0, anneal_shift=0.0, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=1, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, cache_size=875000000.0, categorical_field_ids=[], continuous_field_ids=[], continuous_file=None, continuous_file_column=None, continuous_file_discretization_b

In [3]:
slice_axis = -2
volume_tm = args.tensor_maps_in[0]
slices = volume_tm.shape[slice_axis]
slice_shape = list(volume_tm.shape)
del slice_shape[slice_axis]
slice_tm = TensorMap(f'slice_{volume_tm.input_name()}', shape=slice_shape)
print(f'Slice name: {slice_tm.name} slice shape: {slice_shape} slices: {slices} original shape: {volume_tm.shape}')
args.tensor_maps_in = [slice_tm]
slice_model = make_multimodal_multitask_model(**args.__dict__)
embed_slice_model = make_hidden_layer_model(slice_model, args.tensor_maps_in, 'embed')
embed_slice_model.summary()

in_volume = Input(shape=volume_tm.shape, name=volume_tm.input_name())
embeddings = []
for i in range(slices):
    if slice_axis == -3 or volume_tm.axes() - slice_axis == 2:
        embeddings.append(embed_slice_model(in_volume[..., i, :, :]))
    elif slice_axis == -2 or volume_tm.axes() - slice_axis == 1:
        embeddings.append(embed_slice_model(in_volume[..., i, :]))
    elif slice_axis == -1 or volume_tm.axes() - slice_axis == 0:
        embeddings.append(embed_slice_model(in_volume[..., i]))
    else:
        raise ValueError(f'Can not handle slice axis {slice_axis} with original shape {volume_tm.shape}')
multimodal_activation = concatenate(embeddings, axis=-1)
for units in args.dense_layers:
    multimodal_activation = Dense(units=units, activation=args.activation)(multimodal_activation)

# build decoders
losses = []
my_metrics = {}
loss_weights = []
output_predictions = {}
tensor_maps_out = args.tensor_maps_out
output_tensor_maps_to_process = tensor_maps_out.copy()
while len(output_tensor_maps_to_process) > 0:
    tm = output_tensor_maps_to_process.pop(0)
    losses.append(tm.loss)
    loss_weights.append(tm.loss_weight)
    my_metrics[tm.output_name()] = tm.metrics
    if tm.is_categorical():
        output_predictions[tm] = Dense(units=tm.shape[0], activation='softmax', name=tm.output_name())(multimodal_activation)
    elif tm.axes() == 1:
        output_predictions[tm] = Dense(units=tm.shape[0], activation=tm.activation, name=tm.output_name())(multimodal_activation)

m = Model(inputs=[in_volume], outputs=[output_predictions[tm] for tm in tensor_maps_out])
m.summary()    

Slice name: slice_input_T1_continuous slice shape: [192, 256, 1] slices: 256 original shape: (192, 256, 256, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_slice_input_T1_continuous [(None, 192, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 192, 256, 32) 320         input_slice_input_T1_continuous_c
__________________________________________________________________________________________________
activation (Activation)         (None, 192, 256, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 192, 256, 32) 9248        activation[0][0]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_slice_input_T1_continuous [(None, 192, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 192, 256, 32) 320         input_slice_input_T1_continuous_c
__________________________________________________________________________________________________
activation (Activation)         (None, 192, 256, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 192, 256, 32) 9248        activation[0][0]                 
____________________________________________________________________________________________

Total params: 945,274
Trainable params: 945,274
Non-trainable params: 0
__________________________________________________________________________________________________


In [4]:
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(**args.__dict__)

2020-04-13 13:35:16,997 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/no_id/log_2020-04-13_13-35_0.log.
2020-04-13 13:35:16,998 - arguments:241 - INFO - Command Line was: 
./scripts/tf.sh train --tensors /mnt/disks/brains-all-together/2020-02-11/ --input_tensors t1 --output_tensors sex --training_steps 96 --validation_steps 24 --test_steps 12 --batch_size 1

2020-04-13 13:35:16,999 - arguments:242 - INFO - Total TensorMaps: 474 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, anneal_max=2.0, anneal_rate=0.0, anneal_shift=0.0, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=1, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, cache_size=875000000.0, categorical_field_ids=[], continuous_field_ids=[], continuous_file=None, continuous_file_column=None, continuous_file_discretization_b

In [ ]:
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
m.compile(optimizer=opt, loss=losses, loss_weights=loss_weights, metrics=my_metrics)
train_model_from_generators(m, generate_train, generate_valid, args.training_steps, args.validation_steps, 
                            args.batch_size, args.epochs, args.patience, args.output_folder, args.id, 
                            args.inspect_model, args.inspect_show_labels)